In [13]:
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
positive_dialogs_df = pd.read_excel(r'sentiment_train.xlsx', sheet_name='Positive', header=None, names=['dialog'])
negative_dialogs_df = pd.read_excel(r'sentiment_train.xlsx', sheet_name='Negative', header=None, names=['dialog'])

подключаем модуль предварительной обработки текста

In [3]:
import wordproc as wp

загружаем все морфологические данные по русскому языку
в морфологические данные добавлены словоформы специфичные для этих диалогов, а именно
инет
херня
роутер
рукожопый
интерактивное
разблокировка
куку

In [4]:
lexdata=wp.wordprocessing   
lexdata.lemmatizator = wp.load_lemmatizator('lemmatizator20191211.zip')
lexdata.slovoStop = wp.load_stopSlova('stopword.sts')

создаем список обработанных текстов и список меток 1 - позитив, 2- негатив
В этой модели идет общая классификация разговоров на негативный или позитивный.
Можно построить оценки 
1. положительного или негативного диалога сотрудника
2. оценка того положительно или отрицательно закончиться диалог с клиетом по первым фразам клиента
3. и.д.


In [5]:
text_process=[]
labels=[]

In [6]:
for f_text in positive_dialogs_df['dialog']:
    text_process.append(wp.lemmatizatorPred(
                        wp.tocenizator(f_text, lexdata.slovoStop, wp.rulls()),
                        lexdata.lemmatizator))
    labels.append(1)

In [7]:
for f_text in negative_dialogs_df['dialog']:
    text_process.append(wp.lemmatizatorPred(
        wp.tocenizator(f_text, lexdata.slovoStop, wp.rulls()),
        lexdata.lemmatizator))
    labels.append(2)

Строим частотные вектора текстов TF-IDF

In [8]:
vectorizer = TfidfVectorizer(max_features=500, min_df=1, max_df=0.7)
text_vect = vectorizer.fit_transform(text_process).toarray()


В приведенном выше коде мы определяем, что max_features должно быть 500, что означает, что он использует только 500 наиболее часто встречающихся слов для создания набора векторов с набором слов. Значение взято с избытком т.к. выборка очень маленькая и придется брать для рассмотрения все слова.  Слова, которые встречаются редко, не очень полезны для классификации. Но на маленькой выборке они почти все редкие.
Аналогично, max_df указывает, что используются только те слова, которые встречаются в максимум 70% документов. Слова, встречающиеся во всех документах, слишком распространены и не очень полезны для классификации. Точно так же для min-df установлено значение 1, которое показывает, что оно включает слова, встречающиеся как минимум в 1 документах.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(text_vect,
                                                    labels, test_size=0.2,random_state=0)

Подключаем алгоритм "Случайный лес"

In [16]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=30, random_state=0)
text_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [17]:
predictions = text_classifier.predict(X_test)
print('accuracy = ',accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

accuracy =  1.0
[[2 0]
 [0 2]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2

   micro avg       1.00      1.00      1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



 Классификация методом линейных опорных векторов.

In [20]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train, y_train)
predictionsclf = clf.predict(X_test)
print('accuracy = ',accuracy_score(y_test, predictionsclf))
print(confusion_matrix(y_test, predictionsclf))
print(classification_report(y_test, predictionsclf))

accuracy =  1.0
[[2 0]
 [0 2]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2

   micro avg       1.00      1.00      1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



In [ ]:
Несмотря на 100% показателя классификации верить им нельзя т.к. выборка очень маленькая.